In [1]:
!pip install pyngrok


In [2]:
from pyngrok import ngrok, conf


conf.get_default().auth_token = '2fswxRRYlSMZnWgYzS3SOXi44N2_4pJ1SiQ6LPWrt3zoqyrTF'


public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

 * ngrok tunnel "https://9cf3-34-106-45-59.ngrok-free.app" -> "http://127.0.0.1:5000"


**Cleaning and training**

In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import matplotlib
import plotly.express as px

def data_clean(data1):
  matplotlib.rcParams['font.sans-serif'] = ['Simsun']
  matplotlib.rcParams['font.family'] ='sans-serif'

  matplotlib.rcParams['axes.unicode_minus'] = False


  author_names = data1['author'].str.split(pat=':').str[1]
  all_authors= author_names.str.split(pat=',')#maximum authors = 3

  data1['author'] = all_authors.str[0]
  data1['author1'] = all_authors.str[1]
  data1['author2'] = all_authors.str[2]


  narrator_names = data1['narrator'].str.split(pat=':').str[1]
  all_narrators= narrator_names.str.split(pat=',')#narrators = 3

  data1['narrator'] = all_narrators.str[0]
  data1['narrator1'] = all_narrators.str[1]
  data1['narrator2'] = all_narrators.str[2]




  data1['author'] =  data1['author'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')

  data1['author1'] =  data1['author1'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')
  data1['author2'] =  data1['author2'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')
  data1['narrator'] =  data1['narrator'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')
  data1['narrator1'] =  data1['narrator1'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')
  data1['narrator2'] =  data1['narrator2'].str.split(r"(?<!^)(?=[A-Z])").str.join(' ')

  data1['rating'] = data1['stars'].str.split(' ').str[0]
  data1['no.of.reviews'] = data1['stars'].str.split(' ').str[4].str[5:]

  data1['releasedate'] = list(map(lambda x:pd.to_datetime(x, format = '%d-%m-%y'),data1['releasedate']))

  def hr_to_mins(t):
      t = t.split(' ')
      if t[0] == 'Less':
          return 1
      mins_t = int(t[0])*60 if t[1] == 'hrs' or t[1] == 'hr' else int(t[0])
      mins_t = mins_t + int(t[3]) if len(t) == 5 else mins_t
      return mins_t

  data1['time'] = list(map(lambda x:hr_to_mins(x),data1['time']))

  data1['rating'] = list(map(lambda x: 0 if x == 'Not' else x,data1['rating']))

  data1['rating'] = data1['rating'].astype('float')
  data1['no.of.reviews']= data1['no.of.reviews'].str.replace(pat = ',', repl = '')
  data1['no.of.reviews'] = data1['no.of.reviews'].astype('float')



  data1['language'] = data1['language'].str.title()


  cols = data1.columns.tolist()
  cols = ['name','author','author1','author2','narrator','narrator1','narrator2','language','time','price','releasedate','rating','no.of.reviews','stars']
  data1 = data1[cols]

  data1.rename(columns={'no.of.reviews':'no.of.ratings'}, inplace=True)
  data1.drop(columns=['stars'])

  data1.dtypes
  data1['price'] = data1['price'].str.replace(pat = 'free', repl = '0', case = False)

  data1['price']= data1['price'].str.replace(pat = ',', repl = '', regex=True)

  data1['price'] = data1['price'].astype(float)



  z_scores = np.abs((data1['price'] - data1['price'].mean()) / data1['price'].std())
  threshold = 3
  data1 = data1[z_scores <= threshold]


  z_scores = np.abs((data1['time'] - data1['time'].mean()) / data1['time'].std())
  threshold = 3
  data1 = data1[z_scores <= threshold]

  z_scores = np.abs((data1['no.of.ratings'] - data1['no.of.ratings'].mean()) / data1['no.of.ratings'].std())
  threshold = 3
  data1 = data1[z_scores <= threshold]

  data1['releasedate'] = pd.to_datetime(data1['releasedate']).dt.year
  return data1

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,BaggingRegressor


import pickle

def data_model(data1):

  data1 = data_clean(data1)
  features = ['author','narrator','time']
  target = 'price'


  author_encoder = LabelEncoder()
  narrator_encoder = LabelEncoder()

  data1['author'] = author_encoder.fit_transform(data1['author'])
  data1['narrator'] = narrator_encoder.fit_transform(data1['narrator'])

  X = data1[features]
  y = data1[target]




  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  model = RandomForestRegressor()
  model.fit(
      X_train,
      y_train
  )
  y_pred = model.predict(X_test)


  with open('model.pkl', 'wb') as f:
      pickle.dump(model, f)

  with open('author_encoder.pkl', 'wb') as f:
      pickle.dump(author_encoder, f)

  with open('narrator_encoder.pkl', 'wb') as f:
      pickle.dump(narrator_encoder, f)

  return model,author_encoder,narrator_encoder

data1 = pd.read_csv('audible_uncleaned.csv')
data_model(data1)


(RandomForestRegressor(), LabelEncoder(), LabelEncoder())

In [5]:
!pip install flask flask-ngrok


In [6]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import pickle
import os

app = Flask(__name__)
UPLOAD_FOLDER = '/tmp'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER



def load_encoders_and_model():
    if os.path.exists('model.pkl'):
        with open('model.pkl', 'rb') as f:
            model = pickle.load(f)
        with open('author_encoder.pkl', 'rb') as f:
            author_encoder = pickle.load(f)
        with open('narrator_encoder.pkl', 'rb') as f:
            narrator_encoder = pickle.load(f)
    else:
        model, author_encoder, narrator_encoder = None, None, None
    return model, author_encoder, narrator_encoder

model, author_encoder, narrator_encoder = load_encoders_and_model()

template = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Audio Book Price Predictor</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            background-color: #f5f5f5;
            font-family: 'Helvetica Neue', sans-serif;
            color: #333;
        }
        .container {
            margin-top: 50px;
        }
        .card {
            border: none;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .card-header, .btn {
            background-color: #333;
            color: white;
        }
        .result, .error, .success {
            margin-top: 20px;
            font-weight: bold;
        }'
        .result {
            color: green;
        }
        .error {
            color: red;
        }
        .success {
            color: blue;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="card">
            <div class="card-header text-center">
                <h1>Audio Book Price Suggestion</h1>
            </div>
            <div class="card-body">
                <form method="post" enctype="multipart/form-data">
                    <div class="form-group">
                        <label for="csv_file">Upload CSV to Retrain Model</label>
                        <input type="file" class="form-control-file" id="csv_file" name="csv_file" accept=".csv">
                    </div>
                    <button type="submit" name="retrain" class="btn btn-block">Retrain Model</button>
                </form>
                <form method="post">
                    <div class="form-group">
                        <label for="author">Author</label>
                        <input type="text" class="form-control" id="author" name="author" required>
                    </div>
                    <div class="form-group">
                        <label for="narrator">Narrator</label>
                        <input type="text" class="form-control" id="narrator" name="narrator" required>
                    </div>
                    <div class="form-group">
                        <label for="duration">Duration (mins)</label>
                        <input type="number" class="form-control" id="duration" name="duration" step="0.1" required>
                    </div>
                    <button type="submit" name="predict" class="btn btn-block">Suggest</button>
                </form>
                {% if result %}
                <div class="alert alert-info result">
                    <strong>Suggested Price:</strong> {{ result }}
                </div>
                {% endif %}
                {% if success_message %}
                <div class="alert alert-info success">
                    <strong>Success:</strong> {{ success_message }}
                </div>
                {% endif %}
                {% if error_message %}
                <div class="alert alert-danger error">
                    <strong>Error:</strong> {{ error_message }}
                </div>
                {% endif %}
            </div>
        </div>
    </div>
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.2/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>

'''

@app.route('/', methods=['GET', 'POST'])
def index():
    global model, author_encoder, narrator_encoder
    result = None
    error_message = None
    success_message = None
    if request.method == 'POST':
        if 'predict' in request.form:
            author = request.form.get('author')
            narrator = request.form.get('narrator')
            duration = float(request.form.get('duration'))
            if model is None or author_encoder is None or narrator_encoder is None:
                error_message = "Model not trained yet. Please upload a CSV file to train the model."
            else:
                try:
                    author_encoded = author_encoder.transform([author])[0]
                    narrator_encoded = narrator_encoder.transform([narrator])[0]
                    features = [[author_encoded, narrator_encoded, duration]]
                    prediction = model.predict(features)
                    result =f'{prediction[0]} rs'
                except ValueError as e:
                    error_message = f"Invalid input: {e}"
        elif 'retrain' in request.form and 'csv_file' in request.files:
            file = request.files.get('csv_file')
            if file:
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], 'uploaded_file.csv'))
                try:
                    data = pd.read_csv(os.path.join(app.config['UPLOAD_FOLDER'], 'uploaded_file.csv'))
                    model, author_encoder, narrator_encoder = data_model(data)
                    success_message = "Model retrained successfully."
                except Exception as e:
                    error_message = f"Failed to retrain model: {e}"

    return render_template_string(template, result=result, error_message=error_message, success_message = success_message)


port = 5000
public_url = ngrok.connect(port)
print(f" * Ngrok URL: {public_url}")
app.run(port=port)


 * Ngrok URL: NgrokTunnel: "https://dda6-34-106-45-59.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:01:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:01:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:01:34] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:02:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:07:58] "POST / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:08:55] "POST / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:08:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:08:56] "POST / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:09:08] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2024 01:11:04] "POST / HTTP/1.1" 200 -
